In [ ]:
# This is scratch to do maximum in 10-days intervals. 

In [1]:
import csv
import numpy as np
import pandas as pd
# import geopandas as gpd
from IPython.display import Image
# from shapely.geometry import Point, Polygon
from math import factorial
import scipy
import scipy.signal
import os, os.path

from datetime import date
import datetime
import time

from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn.linear_model import LinearRegression
from patsy import cr

# from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sb

import sys

sys.path.append('/Users/hn/Documents/00_GitHub/Ag/remote_sensing/python/')
import remote_sensing_core as rc
import remote_sensing_core as rcp

start_time = time.time()

In [2]:
data_dir = "/Users/hn/Documents/01_research_data" + \
           "/remote_sensing/01_NDVI_TS/00_Eastern_WA_withYear/"

param_dir = "/Users/hn/Documents/00_GitHub/Ag/remote_sensing/parameters/"

# Parameters

In [3]:
####################################################################################
###
###      Parameters                   
###
####################################################################################

irrigated_only = 0
SF_year = 2017
indeks = "EVI"

regular_window_size = 10

# Read the data

In [4]:
f_name = "Eastern_WA_" + str(SF_year) + "_70cloud_selectors.csv"
a_df = pd.read_csv(data_dir + f_name, low_memory=False)

##################################################################
##################################################################
####
####  plots has to be exact. So, we need 
####  to filter out NASS, and filter by last survey date
####
##################################################################
##################################################################

a_df = a_df[a_df['county']== "Grant"] # Filter Grant
# a_df = rc.filter_out_NASS(a_df) # Toss NASS
# a_df = rc.filter_by_lastSurvey(a_df, year = SF_year) # filter by last survey date
a_df['SF_year'] = SF_year


# Functions

# Get a field's data

In [5]:
a_df.reset_index(drop=True, inplace=True)
a_df_1 = a_df[a_df.ID == a_df.ID[0]]
a_df_1.shape

(319, 26)

In [6]:
a_df_1 = rc.initial_clean_EVI(a_df_1)
# a_df_1.sort_values(by=['system_start_time'], inplace=True)
a_df_1.sort_values(by=['image_year', 'doy'], inplace=True)
a_df_1 = rc.correct_timeColumns_dataTypes(a_df_1)

a_df_1.reset_index(drop=True, inplace=True)
print(a_df_1.shape)
a_df_1.head(2)

(66, 26)


,ID,Acres,BSI,county,CropGrp,CropTyp,DataSrc,doy,EVI,ExctAcr,...,Notes,PSRI,RtCrpTy,Shap_Ar,Shp_Lng,system_start_time,TRS,image_year,B8,SF_year
0,115184_WSDA_SF_2017,72.0,-0.368046,Grant,Orchard,apple,wsda,275,0.417561,72.371199,...,NaN,-0.094055,NaN,292875.850894,3308.620365,1.475434e+09,T13R24E4,2016,0.196237,2017
1,115184_WSDA_SF_2017,72.0,-0.413791,Grant,Orchard,apple,wsda,295,0.741738,72.371199,...,NaN,-0.072641,NaN,292875.850894,3308.620365,1.477162e+09,T13R24E4,2016,0.335369,2017


In [7]:
regularized_TS = rc.regularize_movingWindow_windowSteps_18Months(a_df_1, \
                                                                 SF_yr = SF_year, \
                                                                 idks = indeks, \
                                                                 window_size=10)
print(regularized_TS.shape)

(54, 20)


In [10]:
regularized_TS.head(2)

,ID,Acres,county,CropGrp,CropTyp,DataSrc,ExctAcr,IntlSrD,Irrigtn,LstSrvD,Notes,RtCrpTy,Shap_Ar,Shp_Lng,TRS,image_year,SF_year,doy,EVI,system_start_time
0,115184_WSDA_SF_2017,72.0,Grant,Orchard,apple,wsda,72.371199,2006/08/31,micro-sprinkler,2017/04/24,nan,nan,292875.850894,3308.620365,T13R24E4,2016,2017,274,0.417561,NaN
1,115184_WSDA_SF_2017,72.0,Grant,Orchard,apple,wsda,72.371199,2006/08/31,micro-sprinkler,2017/04/24,nan,nan,292875.850894,3308.620365,T13R24E4,2016,2017,284,-2,NaN


In [ ]:
a_field_df_2017 = a_df_1[a_df_1.image_year == 2017]
print(a_df_1.shape)
print(a_field_df_2017.shape)

In [ ]:
oneYr_regul = regularize_movingWindow_windowSteps_12Months(a_field_df = a_field_df_2017, \
                                                           SF_yr = 2017, \
                                                           idks =" EVI", \
                                                           window_size = 10)
oneYr_regul.shape

In [ ]:
oneYr_regul.shape

In [ ]:
outName = "/Users/hn/Documents/01_research_data/remote_sensing/test_data/a_regularized_TS.csv"
regularized_TS.to_csv(outName, index=False)

In [11]:
regularized_TS.head(2)

,ID,Acres,county,CropGrp,CropTyp,DataSrc,ExctAcr,IntlSrD,Irrigtn,LstSrvD,Notes,RtCrpTy,Shap_Ar,Shp_Lng,TRS,image_year,SF_year,doy,EVI,system_start_time
0,115184_WSDA_SF_2017,72.0,Grant,Orchard,apple,wsda,72.371199,2006/08/31,micro-sprinkler,2017/04/24,nan,nan,292875.850894,3308.620365,T13R24E4,2016,2017,274,0.417561,NaN
1,115184_WSDA_SF_2017,72.0,Grant,Orchard,apple,wsda,72.371199,2006/08/31,micro-sprinkler,2017/04/24,nan,nan,292875.850894,3308.620365,T13R24E4,2016,2017,284,-2,NaN


In [12]:
regularized_TS.shape

(54, 20)

In [17]:
nrows = 54 * 10
reg_cols = ['ID', 'Acres', 'county', 'CropGrp', 'CropTyp',
            'DataSrc', 'ExctAcr', 'IntlSrD', 'Irrigtn', 'LstSrvD', 'Notes',
            'RtCrpTy', 'Shap_Ar', 'Shp_Lng', 'TRS', 'image_year', 
            'SF_year', 'doy', indeks]
output_df = pd.DataFrame(data = None,
                         index = np.arange(nrows), 
                         columns = reg_cols)

In [18]:
output_df.shape

(540, 19)

# Create Aeolus Enviornment, and see things works

In [9]:
first_10_IDs = a_df.ID.unique()[:10]
an_EE_TS = a_df[a_df.ID.isin(first_10_IDs) ]

In [13]:
indeks = "EVI"
county = "Grant"
SF_year = 2017
regular_window_size = 10

In [24]:
########################################################################################

an_EE_TS = an_EE_TS[an_EE_TS['county'] == county] # Filter Grant
an_EE_TS['SF_year'] = SF_year

########################################################################################

# output_dir = "/data/hydro/users/Hossein/remote_sensing/02_Regularized_TS/"

# os.makedirs(output_dir, exist_ok=True)
########################################################################################

if (indeks == "EVI"):
    an_EE_TS = rc.initial_clean_EVI(an_EE_TS)
else:
    an_EE_TS = rc.initial_clean_NDVI(an_EE_TS)

an_EE_TS.head(2)

###
### List of unique polygons
###
polygon_list = an_EE_TS['ID'].unique()
print(len(polygon_list))

########################################################################################
###
###  initialize output data. all polygons in this case
###  will have the same length. 
###  9 steps in the first three months, followed by 36 points in the full year,
###  9 months in the last year
###
reg_cols = ['ID', 'Acres', 'county', 'CropGrp', 'CropTyp',
            'DataSrc', 'ExctAcr', 'IntlSrD', 'Irrigtn', 'LstSrvD', 'Notes',
            'RtCrpTy', 'Shap_Ar', 'Shp_Lng', 'TRS', 'image_year', 
            'SF_year', 'doy', indeks]

nrows = 54 * len(polygon_list)
output_df = pd.DataFrame(data = None,
                         index = np.arange(nrows), 
                         columns = reg_cols)
########################################################################################

counter = 0

for a_poly in polygon_list:
    if (counter): # % 100 == 0
        print (counter)
    
    curr_field = an_EE_TS[an_EE_TS['ID']==a_poly].copy()
    ################################################################
    # Sort by DoY (sanitary check)
    curr_field.sort_values(by=['image_year', 'doy'], inplace=True)
    
    curr_field = rc.correct_timeColumns_dataTypes(curr_field)
    curr_field.reset_index(drop=True, inplace=True)
    
    print ("print(curr_field.shape")
    print(curr_field.shape)
    print ("__________________________________________")
    ################################################################
    regularized_TS = rc.regularize_movingWindow_windowSteps_18Months(curr_field, \
                                                                     SF_yr = SF_year, \
                                                                     idks = indeks, \
                                                                     window_size = 10)
    print(regularized_TS.shape)

    ################################################################
    row_pointer = 54 * counter
    output_df[row_pointer: row_pointer+54] = regularized_TS.values
    counter += 1




10
print(curr_field.shape
(66, 26)
__________________________________________
(54, 19)
1
print(curr_field.shape
(66, 26)
__________________________________________
(54, 19)
2
print(curr_field.shape
(66, 26)
__________________________________________
(54, 19)
3
print(curr_field.shape
(63, 26)
__________________________________________
(54, 19)
4
print(curr_field.shape
(61, 26)
__________________________________________
(54, 19)
5
print(curr_field.shape
(63, 26)
__________________________________________
(54, 19)
6
print(curr_field.shape
(63, 26)
__________________________________________
(54, 19)
7
print(curr_field.shape
(61, 26)
__________________________________________
(54, 19)
8
print(curr_field.shape
(60, 26)
__________________________________________
(54, 19)
9
print(curr_field.shape
(63, 26)
__________________________________________
(54, 19)


In [25]:
regularized_TS.values.shape

(54, 19)

In [26]:
output_df[row_pointer: row_pointer+54].shape

(54, 19)

In [27]:
row_pointer

486

In [28]:
output_df.shape

(540, 19)

In [30]:
output_df.head(2)

,ID,Acres,county,CropGrp,CropTyp,DataSrc,ExctAcr,IntlSrD,Irrigtn,LstSrvD,Notes,RtCrpTy,Shap_Ar,Shp_Lng,TRS,image_year,SF_year,doy,EVI
0,115184_WSDA_SF_2017,72,Grant,Orchard,apple,wsda,72.3712,2006/08/31,micro-sprinkler,2017/04/24,nan,nan,292876,3308.62,T13R24E4,2016,2017,274,0.417561
1,115184_WSDA_SF_2017,72,Grant,Orchard,apple,wsda,72.3712,2006/08/31,micro-sprinkler,2017/04/24,nan,nan,292876,3308.62,T13R24E4,2016,2017,284,-2


In [ ]:
print (a_df_1.system_start_time.iloc[0])
print (time.strftime('%Y-%m-%d', time.localtime(a_df_1.system_start_time.iloc[0])))


print ("Convert Epoch to datetime format")
print (datetime.datetime.fromtimestamp(a_df_1.system_start_time.iloc[0]))

# Convert Epoch to DoY
print ("___________________________________________")
print ("")
print ("Convert Epoch to DoY")
print ( (datetime.datetime.fromtimestamp(a_df_1.system_start_time.iloc[0])).timetuple().tm_yday  )

print ("___________________________________________")
print ("")
print ("difference number of days")
print ((date(2003,11,22) - date(2002,10,20)).days)

In [ ]:
time.localtime(a_df_1.system_start_time.iloc[0])

In [ ]:
# datetime.datetime(2016, 1, 1) + datetime.timedelta(275 - 1)

In [ ]:
# im_yr_sotred = a_df_1.copy()
# epoch_sorted = a_df_1.copy()
# im_yr_sotred.sort_values(by=['image_year', 'doy'], inplace=True)
# epoch_sorted.sort_values(by=['system_start_time'], inplace=True)
# epoch_sorted.to_csv (r'/Users/hn/Desktop/test/epoch_sorted.csv', index = True, header=True)
# im_yr_sotred.to_csv (r'/Users/hn/Desktop/test/im_yr_sotred.csv', index = True, header=True)
# a_df_1.to_csv (r'/Users/hn/Desktop/test/a_df_1.csv', index = True, header=True)